## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cape Town,ZA,2021-05-11 06:52:59,-33.9258,18.4232,60.01,94,0,2.30,fog
1,1,Mar Del Plata,AR,2021-05-11 06:49:11,-38.0023,-57.5575,46.99,80,6,3.00,clear sky
2,2,Mazatlan,MX,2021-05-11 06:53:13,23.2167,-106.4167,79.00,88,90,2.73,overcast clouds
3,3,Bonavista,CA,2021-05-11 06:53:14,48.6499,-53.1147,41.00,87,100,9.22,overcast clouds
4,4,Port Alfred,ZA,2021-05-11 06:53:14,-33.5906,26.8910,65.10,72,3,8.88,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Mazatlan,MX,2021-05-11 06:53:13,23.2167,-106.4167,79.00,88,90,2.73,overcast clouds
6,6,Itaueira,BR,2021-05-11 06:53:15,-7.6033,-43.0256,75.79,67,100,11.70,overcast clouds
10,10,Faya,SA,2021-05-11 06:53:16,18.3851,42.4509,77.00,27,0,3.44,clear sky
11,11,Acapulco,MX,2021-05-11 06:53:16,16.8634,-99.8901,80.76,77,67,1.68,broken clouds
12,12,Carnarvon,AU,2021-05-11 06:53:16,-24.8667,113.6333,84.20,42,7,8.05,clear sky
21,21,Butaritari,KI,2021-05-11 06:53:19,3.0707,172.7902,81.43,80,70,14.20,moderate rain
22,22,Busselton,AU,2021-05-11 06:53:19,-33.6500,115.3333,75.99,58,89,5.99,overcast clouds
23,23,Namibe,AO,2021-05-11 06:53:20,-15.1961,12.1522,76.71,66,6,7.11,clear sky
24,24,Kavaratti,IN,2021-05-11 06:53:20,10.5669,72.6420,85.80,68,63,3.27,broken clouds
30,30,Naze,JP,2021-05-11 06:53:22,28.3667,129.4833,77.00,94,75,12.66,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                189
City                   189
Country                188
Date                   189
Lat                    189
Lng                    189
Max Temp               189
Humidity               189
Cloudiness             189
Wind Speed             189
Current Description    189
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Mazatlan,MX,79.00,overcast clouds,23.2167,-106.4167,
6,Itaueira,BR,75.79,overcast clouds,-7.6033,-43.0256,
10,Faya,SA,77.00,clear sky,18.3851,42.4509,
11,Acapulco,MX,80.76,broken clouds,16.8634,-99.8901,
12,Carnarvon,AU,84.20,clear sky,-24.8667,113.6333,
21,Butaritari,KI,81.43,moderate rain,3.0707,172.7902,
22,Busselton,AU,75.99,overcast clouds,-33.6500,115.3333,
23,Namibe,AO,76.71,clear sky,-15.1961,12.1522,
24,Kavaratti,IN,85.80,broken clouds,10.5669,72.6420,
30,Naze,JP,77.00,broken clouds,28.3667,129.4833,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

clean_hotel_df.count()

City                   188
Country                188
Max Temp               188
Current Description    188
Lat                    188
Lng                    188
Hotel Name             188
dtype: int64

In [15]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [14]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))